# Conjunto Census

In [4]:
import pandas as pd
import numpy as np
import pickle
from utils import *
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, log_loss
from sklearn.model_selection import StratifiedKFold, cross_validate
from slice_finder import SliceFinder
import warnings
warnings.filterwarnings('ignore')

## Visualizando regiões identificadas para o conjunto Census

In [2]:
with open('./Slices/Census.p', 'rb') as file:
    data = pickle.load(file)
    
    data = sorted(data, key=lambda d: d.size, reverse=True)
    for d in data:
        print(f'Descrição da região:  {d.filters}')
        print(f'Instâncias da região: {d.data_idx}')
        print(f'Tamanho da região:    {d.size}')
        print(f'Efect size da região: {d.effect_size}')
        print(f'Metrica da região:    {d.metric}') # Log Loss
        print('\n')

Descrição da região:  {'Martial Status': [[2]]}
Instâncias da região: Index([    1,     3,     4,     5,     7,     9,    10,    11,    15,    18,
       ...
       32533, 32545, 32547, 32550, 32551, 32552, 32554, 32556, 32557, 32560],
      dtype='int64', length=14065)
Tamanho da região:    14065
Efect size da região: 0.604267951723974
Metrica da região:    0.5572188622193317


Descrição da região:  {'Relationship': [[0]]}
Instâncias da região: Index([    1,     3,     7,     9,    10,    11,    15,    18,    20,    22,
       ...
       32522, 32526, 32532, 32533, 32547, 32550, 32551, 32552, 32554, 32557],
      dtype='int64', length=12463)
Tamanho da região:    12463
Efect size da região: 0.5553358604790886
Metrica da região:    0.5555271649963417


Descrição da região:  {'Country': [[38]], 'Relationship': [[5]]}
Instâncias da região: Index([    5,    37,    50,    67,    82,   113,   125,   137,   188,   208,
       ...
       32347, 32386, 32421, 32479, 32485, 32513, 32528, 32545,

# Avaliando desempenho

In [25]:
adult_data_train = pd.read_csv(
    "data/adult.data",
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "EducationNum", "MartialStatus",
        "Occupation", "Relationship", "Race", "Sex", "CapitalGain", "CapitalLoss",
        "Hoursperweek", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

# drop nan values
adult_data_train = adult_data_train.dropna()

# Encode categorical features
encoders = {}
for column in adult_data_train.columns:
    if adult_data_train.dtypes[column] == np.object_:
        le = LabelEncoder()
        adult_data_train[column] = le.fit_transform(adult_data_train[column])
        encoders[column] = le
        print(column, le.classes_, le.transform(le.classes_))

X_train, y_train = adult_data_train[adult_data_train.columns.difference(["Target"])], adult_data_train["Target"]

Workclass ['Federal-gov' 'Local-gov' 'Private' 'Self-emp-inc' 'Self-emp-not-inc'
 'State-gov' 'Without-pay'] [0 1 2 3 4 5 6]
Education ['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college'] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
MartialStatus ['Divorced' 'Married-AF-spouse' 'Married-civ-spouse'
 'Married-spouse-absent' 'Never-married' 'Separated' 'Widowed'] [0 1 2 3 4 5 6]
Occupation ['Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving'] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Relationship ['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife'] [0 1 2 3 4 5]
Race ['Amer-Indian-Eskimo' 'Asian-Pac-Islander' 'Black' 'Other' 'White'] [0 1 2 3 4]
Sex ['Female' 'Ma

In [3]:
adult_data_train.groupby('Target').describe()

Age                                                    Workclass  \
          count      mean        std   min   25%   50%   75%   max     count   
Target                                                                         
0       22654.0  36.60806  13.464631  17.0  26.0  34.0  45.0  90.0   22654.0   
1        7508.0  43.95911  10.269633  19.0  36.0  43.0  51.0  90.0    7508.0   

                  ... Hoursperweek        Country                            \
            mean  ...          75%   max    count       mean       std  min   
Target            ...                                                         
0       2.189415  ...         40.0  99.0  22654.0  36.300786  6.188487  0.0   
1       2.229222  ...         50.0  99.0   7508.0  36.629329  5.840897  0.0   

                                
         25%   50%   75%   max  
Target                          
0       38.0  38.0  38.0  40.0  
1       38.0  38.0  38.0  40.0  

[2 rows x 112 columns]

In [26]:
adult_data_test = pd.read_csv(
    "data/adult.data",
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "EducationNum", "MartialStatus",
        "Occupation", "Relationship", "Race", "Sex", "CapitalGain", "CapitalLoss",
        "Hoursperweek", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

# drop nan values
adult_data_test = adult_data_test.dropna()

# Encode categorical features
encoders = {}
for column in adult_data_test.columns:
    if adult_data_test.dtypes[column] == np.object_:
        le = LabelEncoder()
        adult_data_test[column] = le.fit_transform(adult_data_test[column])
        encoders[column] = le
        print(column, le.classes_, le.transform(le.classes_))

X_val, y_val = adult_data_test[adult_data_test.columns.difference(["Target"])], adult_data_test["Target"]

Workclass ['Federal-gov' 'Local-gov' 'Private' 'Self-emp-inc' 'Self-emp-not-inc'
 'State-gov' 'Without-pay'] [0 1 2 3 4 5 6]
Education ['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college'] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
MartialStatus ['Divorced' 'Married-AF-spouse' 'Married-civ-spouse'
 'Married-spouse-absent' 'Never-married' 'Separated' 'Widowed'] [0 1 2 3 4 5 6]
Occupation ['Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving'] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Relationship ['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife'] [0 1 2 3 4 5]
Race ['Amer-Indian-Eskimo' 'Asian-Pac-Islander' 'Black' 'Other' 'White'] [0 1 2 3 4]
Sex ['Female' 'Ma

In [27]:
from sklearn.model_selection import cross_val_predict

# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_train, y_train)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_train, y_train, cv=skf, scoring=scoring)

# Previsões no conjunto de treinamento usando validação cruzada
y_train_pred = cross_val_predict(rf_optimized, X_train, y_train, cv=skf)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_train, y_train)

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")

# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Relatório de classificação no conjunto de treinamento
print("\nRelatório de classificação no conjunto de treinamento (validação cruzada):")
print(classification_report(y_train, y_train_pred))

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 479.5881595611572 segundos

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.86 (+/- 0.01)
Recall: 0.62 (+/- 0.02)
F1 Score: 0.69 (+/- 0.02)
Log-loss: 0.30 (+/- 0.01)

Relatório de classificação no conjunto de treinamento (validação cruzada):
              precision    recall  f1-score   support

           0       0.88      0.94      0.91     22654
           1       0.78      0.62      0.69      7508

    accuracy                           0.86     30162
   macro avg       0.83      0.78      0.80     30162
weighted avg       0.86      0.86      0.86     30162


Resultados no conjunto de validação:
Acurácia: 0.90
Log-loss: 0.23

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.91      0.96      0.93     22654
           1       0.86      0.70      0.77      7508

    accuracy                           0.90     30162
   macro avg   

In [5]:
# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_train, y_train)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_train, y_train, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_train, y_train)

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")

# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 475.4234387874603 segundos

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.86 (+/- 0.01)
Recall: 0.62 (+/- 0.02)
F1 Score: 0.69 (+/- 0.02)
Log-loss: 0.30 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.90
Log-loss: 0.23

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.91      0.96      0.93     22654
           1       0.86      0.70      0.77      7508

    accuracy                           0.90     30162
   macro avg       0.88      0.83      0.85     30162
weighted avg       0.89      0.90      0.89     30162



## Aplicando SMOTE a todo o conjunto

In [6]:
# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

#Instanciar SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)

y_slice = y_train
X_slice = X_train

#Balencear região
X_slice_smote, y_slice_smote = smote.fit_resample(X_slice, y_slice)
region_SMOTE = (pd.concat([pd.DataFrame(X_slice_smote, columns=X_slice.columns), pd.Series(y_slice_smote, name='Target')], axis=1))

#Separar novo conjunto
y_SMOTE_data = region_SMOTE['Target']
X_SMOTE_data = region_SMOTE.drop(columns='Target')

# Treina o modelo no conjunto de treinamento aplicado SMOTE
start_time_model = time.time()
rf_optimized = model_train(X_SMOTE_data, y_SMOTE_data)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliar no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_SMOTE_data, y_SMOTE_data)

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print("Tamanho do novo conjunto de dados:", len(X_SMOTE_data), "para o treinamento")
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 697.6612439155579 segundos
Tamanho do novo conjunto de dados: 45308 para o treinamento

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.89 (+/- 0.01)
Recall: 0.90 (+/- 0.01)
F1 Score: 0.89 (+/- 0.01)
Log-loss: 0.25 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.94
Log-loss: 0.16

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96     22654
           1       0.85      0.91      0.88      7508

    accuracy                           0.94     30162
   macro avg       0.91      0.93      0.92     30162
weighted avg       0.94      0.94      0.94     30162



### Aumento de dados guiado por regiões problemáticas

In [7]:
region = adult_data_train.query('MartialStatus==2')
region.groupby('Target').describe()

Age                                                     Workclass  \
         count       mean        std   min   25%   50%   75%   max     count   
Target                                                                         
0       7666.0  41.636968  12.653064  17.0  32.0  40.0  50.0  90.0    7666.0   
1       6399.0  44.127989  10.112189  21.0  37.0  44.0  51.0  90.0    6399.0   

                  ... Hoursperweek       Country                            \
            mean  ...          75%   max   count       mean       std  min   
Target            ...                                                        
0       2.316332  ...         45.0  99.0  7666.0  36.148448  6.315194  0.0   
1       2.235506  ...         50.0  99.0  6399.0  36.614784  5.906077  0.0   

                                
         25%   50%   75%   max  
Target                          
0       38.0  38.0  38.0  40.0  
1       38.0  38.0  38.0  40.0  

[2 rows x 112 columns]

In [8]:
# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define a região
region = 'MartialStatus==2'

# Aplica o SMOTE na região de interesse
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_region = X_train.query(region)  # Seleciona apenas as amostras da região
y_region = y_train[X_region.index]

# Aplica o SMOTE na região selecionada
X_region_smote, y_region_smote = smote.fit_resample(X_region, y_region)

# Combina o conjunto original com o balanceado para formar o novo conjunto de treino
X_SMOTE_data = pd.concat([X_train.drop(X_region.index), pd.DataFrame(X_region_smote, columns=X_train.columns)])
y_SMOTE_data = pd.concat([y_train.drop(X_region.index), pd.Series(y_region_smote)])

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_SMOTE_data, y_SMOTE_data)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_SMOTE_data, y_SMOTE_data) 

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print("Tamanho do novo conjunto de dados:", len(X_SMOTE_data), "para o treinamento")
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 525.3671760559082 segundos
Tamanho do novo conjunto de dados: 31429 para o treinamento

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.85 (+/- 0.01)
Recall: 0.67 (+/- 0.01)
F1 Score: 0.72 (+/- 0.01)
Log-loss: 0.32 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.88
Log-loss: 0.26

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92     22654
           1       0.81      0.69      0.75      7508

    accuracy                           0.88     30162
   macro avg       0.86      0.82      0.84     30162
weighted avg       0.88      0.88      0.88     30162



In [9]:
region = adult_data_train.query('Relationship==0')
region.groupby('Target').describe()

Age                                                     Workclass  \
         count       mean        std   min   25%   50%   75%   max     count   
Target                                                                         
0       6784.0  42.131191  12.643627  17.0  32.0  40.0  51.0  90.0    6784.0   
1       5679.0  44.561719  10.074241  22.0  37.0  44.0  51.0  90.0    5679.0   

                  ... Hoursperweek       Country                            \
            mean  ...          75%   max   count       mean       std  min   
Target            ...                                                        
0       2.330041  ...         48.0  99.0  6784.0  36.283903  6.090564  0.0   
1       2.244409  ...         50.0  99.0  5679.0  36.684804  5.727642  0.0   

                                
         25%   50%   75%   max  
Target                          
0       38.0  38.0  38.0  40.0  
1       38.0  38.0  38.0  40.0  

[2 rows x 112 columns]

In [10]:
# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define a região
region = 'Relationship==0'

# Aplica o SMOTE na região de interesse
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_region = X_train.query(region)  # Seleciona apenas as amostras da região
y_region = y_train[X_region.index]

# Aplica o SMOTE na região selecionada
X_region_smote, y_region_smote = smote.fit_resample(X_region, y_region)

# Combina o conjunto original com o balanceado para formar o novo conjunto de treino
X_SMOTE_data = pd.concat([X_train.drop(X_region.index), pd.DataFrame(X_region_smote, columns=X_train.columns)])
y_SMOTE_data = pd.concat([y_train.drop(X_region.index), pd.Series(y_region_smote)])

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_SMOTE_data, y_SMOTE_data)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_SMOTE_data, y_SMOTE_data) 

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print("Tamanho do novo conjunto de dados:", len(X_SMOTE_data), "para o treinamento")
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 510.6092712879181 segundos
Tamanho do novo conjunto de dados: 31267 para o treinamento

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.86 (+/- 0.01)
Recall: 0.68 (+/- 0.01)
F1 Score: 0.73 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.91
Log-loss: 0.21

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.93      0.96      0.94     22654
           1       0.87      0.77      0.82      7508

    accuracy                           0.91     30162
   macro avg       0.90      0.87      0.88     30162
weighted avg       0.91      0.91      0.91     30162



In [12]:
region = adult_data_train.query('CapitalGain==0 and Relationship==5')
region.groupby('Target').describe()

Age                                                     Workclass  \
        count       mean        std   min   25%   50%   75%   max     count   
Target                                                                        
0       666.0  38.833333  11.622670  18.0  29.0  38.0  47.0  77.0     666.0   
1       564.0  40.432624   9.661816  21.0  33.0  40.0  47.0  90.0     564.0   

                  ... Hoursperweek       Country                            \
            mean  ...          75%   max   count       mean       std  min   
Target            ...                                                        
0       2.213213  ...         40.0  99.0   666.0  35.848348  6.708729  1.0   
1       2.154255  ...         40.0  80.0   564.0  35.987589  7.293019  1.0   

                                
         25%   50%   75%   max  
Target                          
0       38.0  38.0  38.0  39.0  
1       38.0  38.0  38.0  40.0  

[2 rows x 112 columns]

In [13]:
# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define a região
region = 'CapitalGain==0 and Relationship==5'

# Aplica o SMOTE na região de interesse
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_region = X_train.query(region)  # Seleciona apenas as amostras da região
y_region = y_train[X_region.index]

# Aplica o SMOTE na região selecionada
X_region_smote, y_region_smote = smote.fit_resample(X_region, y_region)

# Combina o conjunto original com o balanceado para formar o novo conjunto de treino
X_SMOTE_data = pd.concat([X_train.drop(X_region.index), pd.DataFrame(X_region_smote, columns=X_train.columns)])
y_SMOTE_data = pd.concat([y_train.drop(X_region.index), pd.Series(y_region_smote)])

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_SMOTE_data, y_SMOTE_data)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_SMOTE_data, y_SMOTE_data) 

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print("Tamanho do novo conjunto de dados:", len(X_SMOTE_data), "para o treinamento")
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 350.718407869339 segundos
Tamanho do novo conjunto de dados: 30264 para o treinamento

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.86 (+/- 0.01)
Recall: 0.64 (+/- 0.01)
F1 Score: 0.70 (+/- 0.01)
Log-loss: 0.31 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.93
Log-loss: 0.18

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.94      0.97      0.95     22654
           1       0.90      0.80      0.85      7508

    accuracy                           0.93     30162
   macro avg       0.92      0.89      0.90     30162
weighted avg       0.93      0.93      0.93     30162



In [14]:
region = adult_data_train.query('Education==14 and CapitalGain==0')
region.groupby('Target').describe()

Age                                                       Workclass  \
        count       mean        std   min    25%   50%    75%   max     count   
Target                                                                          
0       128.0  42.945312  15.263207  25.0  29.75  40.0  53.25  88.0     128.0   
1       275.0  44.040000   9.629365  26.0  37.00  43.0  50.00  80.0     275.0   

                  ... Hoursperweek       Country                            \
            mean  ...          75%   max   count       mean       std  min   
Target            ...                                                        
0       2.515625  ...         50.0  99.0   128.0  35.585938  7.725128  1.0   
1       2.654545  ...         55.0  99.0   275.0  36.021818  6.602330  3.0   

                                
         25%   50%   75%   max  
Target                          
0       38.0  38.0  38.0  38.0  
1       38.0  38.0  38.0  38.0  

[2 rows x 112 columns]

In [15]:
# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define a região
region = 'Education==14 and CapitalGain==0'

# Aplica o SMOTE na região de interesse
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_region = X_train.query(region)  # Seleciona apenas as amostras da região
y_region = y_train[X_region.index]

# Aplica o SMOTE na região selecionada
X_region_smote, y_region_smote = smote.fit_resample(X_region, y_region)

# Combina o conjunto original com o balanceado para formar o novo conjunto de treino
X_SMOTE_data = pd.concat([X_train.drop(X_region.index), pd.DataFrame(X_region_smote, columns=X_train.columns)])
y_SMOTE_data = pd.concat([y_train.drop(X_region.index), pd.Series(y_region_smote)])

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_SMOTE_data, y_SMOTE_data)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_SMOTE_data, y_SMOTE_data) 

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print("Tamanho do novo conjunto de dados:", len(X_SMOTE_data), "para o treinamento")
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 585.828736782074 segundos
Tamanho do novo conjunto de dados: 30309 para o treinamento

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.86 (+/- 0.00)
Recall: 0.61 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.89
Log-loss: 0.24

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93     22654
           1       0.85      0.68      0.76      7508

    accuracy                           0.89     30162
   macro avg       0.88      0.82      0.84     30162
weighted avg       0.89      0.89      0.89     30162



In [16]:
region = adult_data_train.query('CapitalGain==0 and EducationNum==15')
region.groupby('Target').describe()

Age                                                       Workclass  \
        count       mean        std   min    25%   50%    75%   max     count   
Target                                                                          
0       128.0  42.945312  15.263207  25.0  29.75  40.0  53.25  88.0     128.0   
1       275.0  44.040000   9.629365  26.0  37.00  43.0  50.00  80.0     275.0   

                  ... Hoursperweek       Country                            \
            mean  ...          75%   max   count       mean       std  min   
Target            ...                                                        
0       2.515625  ...         50.0  99.0   128.0  35.585938  7.725128  1.0   
1       2.654545  ...         55.0  99.0   275.0  36.021818  6.602330  3.0   

                                
         25%   50%   75%   max  
Target                          
0       38.0  38.0  38.0  38.0  
1       38.0  38.0  38.0  38.0  

[2 rows x 112 columns]

In [17]:
# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define a região
region = 'CapitalGain==0 and EducationNum==15'

# Aplica o SMOTE na região de interesse
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_region = X_train.query(region)  # Seleciona apenas as amostras da região
y_region = y_train[X_region.index]

# Aplica o SMOTE na região selecionada
X_region_smote, y_region_smote = smote.fit_resample(X_region, y_region)

# Combina o conjunto original com o balanceado para formar o novo conjunto de treino
X_SMOTE_data = pd.concat([X_train.drop(X_region.index), pd.DataFrame(X_region_smote, columns=X_train.columns)])
y_SMOTE_data = pd.concat([y_train.drop(X_region.index), pd.Series(y_region_smote)])

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_SMOTE_data, y_SMOTE_data)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_SMOTE_data, y_SMOTE_data, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_SMOTE_data, y_SMOTE_data) 

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print("Tamanho do novo conjunto de dados:", len(X_SMOTE_data), "para o treinamento")
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 533.9315240383148 segundos
Tamanho do novo conjunto de dados: 30309 para o treinamento

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.86 (+/- 0.00)
Recall: 0.61 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.89
Log-loss: 0.24

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93     22654
           1       0.85      0.68      0.76      7508

    accuracy                           0.89     30162
   macro avg       0.88      0.82      0.84     30162
weighted avg       0.89      0.89      0.89     30162



### remoção da região problemática

In [12]:
def import_census():
    adult_data_train = pd.read_csv(
    "data/adult.data",
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "EducationNum", "MartialStatus",
        "Occupation", "Relationship", "Race", "Sex", "CapitalGain", "CapitalLoss",
        "Hoursperweek", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

    # drop nan values
    adult_data_train = adult_data_train.dropna()

    # Encode categorical features
    encoders = {}
    for column in adult_data_train.columns:
        if adult_data_train.dtypes[column] == np.object_:
            le = LabelEncoder()
            adult_data_train[column] = le.fit_transform(adult_data_train[column])
            encoders[column] = le
            #print(column, le.classes_, le.transform(le.classes_))

    X_train, y_train = adult_data_train[adult_data_train.columns.difference(["Target"])], adult_data_train["Target"]
    
    return adult_data_train, X_train, y_train

In [15]:
adult_data_train_, X_train, y_train = import_census()

# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define a região para remoção
region = 'CapitalGain==0 and Relationship==5'

# Identifica e remove as amostras da região especificada
region_remove_idx = adult_data_train_.query(region).index
adult_data_train_.drop(region_remove_idx, errors='ignore', inplace=True)

# Garante que as colunas do conjunto de treino permaneçam consistentes
X_slice = adult_data_train_.drop(columns=['Target'])
y_slice = adult_data_train_['Target']

# Reindexa o conjunto de validação para garantir consistência com o treino
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_slice, y_slice)

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print(len(X_slice))
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 400.81226563453674 segundos
28932

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.87 (+/- 0.00)
Recall: 0.61 (+/- 0.02)
F1 Score: 0.69 (+/- 0.02)
Log-loss: 0.29 (+/- 0.00)

Resultados no conjunto de validação:
Acurácia: 0.88
Log-loss: 0.26

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92     22654
           1       0.84      0.62      0.71      7508

    accuracy                           0.88     30162
   macro avg       0.86      0.79      0.82     30162
weighted avg       0.87      0.88      0.87     30162



In [16]:
adult_data_train_, X_train, y_train = import_census()

# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define a região para remoção
region = 'Education==14 and CapitalGain==0' 

# Identifica e remove as amostras da região especificada
region_remove_idx = adult_data_train_.query(region).index
adult_data_train_.drop(region_remove_idx, errors='ignore', inplace=True)

# Garante que as colunas do conjunto de treino permaneçam consistentes
X_slice = adult_data_train_.drop(columns=['Target'])
y_slice = adult_data_train_['Target']

# Reindexa o conjunto de validação para garantir consistência com o treino
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_slice, y_slice)

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print(len(X_slice))
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 590.8934690952301 segundos
29759

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.86 (+/- 0.01)
Recall: 0.61 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.92
Log-loss: 0.19

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     22654
           1       0.89      0.76      0.82      7508

    accuracy                           0.92     30162
   macro avg       0.91      0.87      0.89     30162
weighted avg       0.92      0.92      0.92     30162



In [17]:
adult_data_train_, X_train, y_train = import_census()

# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define a região para remoção
region = 'CapitalGain==0 and EducationNum==15' 

# Identifica e remove as amostras da região especificada
region_remove_idx = adult_data_train_.query(region).index
adult_data_train_.drop(region_remove_idx, errors='ignore', inplace=True)

# Garante que as colunas do conjunto de treino permaneçam consistentes
X_slice = adult_data_train_.drop(columns=['Target'])
y_slice = adult_data_train_['Target']

# Reindexa o conjunto de validação para garantir consistência com o treino
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_slice, y_slice)

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print(len(X_slice))
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 576.6541233062744 segundos
29759

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.86 (+/- 0.01)
Recall: 0.61 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.92
Log-loss: 0.19

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     22654
           1       0.89      0.76      0.82      7508

    accuracy                           0.92     30162
   macro avg       0.91      0.87      0.89     30162
weighted avg       0.92      0.92      0.92     30162



In [18]:
adult_data_train_, X_train, y_train = import_census()

# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define a região para remoção
region = 'Country==38 and Relationship==5' 

# Identifica e remove as amostras da região especificada
region_remove_idx = adult_data_train_.query(region).index
adult_data_train_.drop(region_remove_idx, errors='ignore', inplace=True)

# Garante que as colunas do conjunto de treino permaneçam consistentes
X_slice = adult_data_train_.drop(columns=['Target'])
y_slice = adult_data_train_['Target']

# Reindexa o conjunto de validação para garantir consistência com o treino
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_slice, y_slice)

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print(len(X_slice))
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 582.5252411365509 segundos
28911

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.87 (+/- 0.01)
Recall: 0.61 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.29 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.90
Log-loss: 0.22

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.91      0.97      0.93     22654
           1       0.87      0.70      0.77      7508

    accuracy                           0.90     30162
   macro avg       0.89      0.83      0.85     30162
weighted avg       0.90      0.90      0.89     30162



In [19]:
adult_data_train_, X_train, y_train = import_census()

# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Define a região para remoção
region = 'Relationship==5 and Race==4'


# Identifica e remove as amostras da região especificada
region_remove_idx = adult_data_train_.query(region).index
adult_data_train_.drop(region_remove_idx, errors='ignore', inplace=True)

# Garante que as colunas do conjunto de treino permaneçam consistentes
X_slice = adult_data_train_.drop(columns=['Target'])
y_slice = adult_data_train_['Target']

# Reindexa o conjunto de validação para garantir consistência com o treino
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_slice, y_slice)

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print(len(X_slice))
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 883.7780649662018 segundos
28973

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.87 (+/- 0.01)
Recall: 0.61 (+/- 0.02)
F1 Score: 0.69 (+/- 0.02)
Log-loss: 0.29 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.90
Log-loss: 0.22

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94     22654
           1       0.89      0.70      0.78      7508

    accuracy                           0.90     30162
   macro avg       0.90      0.84      0.86     30162
weighted avg       0.90      0.90      0.90     30162



### remoção de região aleatória

In [21]:
adult_data_train_, X_train, y_train = import_census()


# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Remove aleatoriamente uma quantidade definida de linhas
remove_rows = adult_data_train_.sample(n=len(adult_data_train_.query('Country==38 and Relationship==5').index), random_state=42).index
adult_data_train_.drop(remove_rows, inplace=True)

# Prepara os dados de treino após a remoção
y_slice = adult_data_train_['Target']
X_slice = adult_data_train_.drop(columns=['Target'])

# Garante que as colunas do conjunto de validação estejam alinhadas com o conjunto de treino
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_slice, y_slice)

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print(len(X_slice))
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 703.7921741008759 segundos
28911

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.86 (+/- 0.01)
Recall: 0.62 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.90
Log-loss: 0.22

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94     22654
           1       0.87      0.72      0.79      7508

    accuracy                           0.90     30162
   macro avg       0.89      0.84      0.86     30162
weighted avg       0.90      0.90      0.90     30162



In [22]:
adult_data_train_, X_train, y_train = import_census()


# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Remove aleatoriamente uma quantidade definida de linhas
remove_rows = adult_data_train_.sample(n=len(adult_data_train_.query( 'CapitalGain==0 and EducationNum==15').index), random_state=42).index
adult_data_train_.drop(remove_rows, inplace=True)

# Prepara os dados de treino após a remoção
y_slice = adult_data_train_['Target']
X_slice = adult_data_train_.drop(columns=['Target'])

# Garante que as colunas do conjunto de validação estejam alinhadas com o conjunto de treino
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_slice, y_slice)

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print(len(X_slice))
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 596.66472697258 segundos
29759

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.86 (+/- 0.00)
Recall: 0.62 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.90
Log-loss: 0.23

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.91      0.96      0.94     22654
           1       0.87      0.71      0.78      7508

    accuracy                           0.90     30162
   macro avg       0.89      0.84      0.86     30162
weighted avg       0.90      0.90      0.90     30162



In [23]:
adult_data_train_, X_train, y_train = import_census()


# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Remove aleatoriamente uma quantidade definida de linhas
remove_rows = adult_data_train_.sample(n=len(adult_data_train_.query('CapitalGain==0 and Relationship==5').index), random_state=42).index
adult_data_train_.drop(remove_rows, inplace=True)

# Prepara os dados de treino após a remoção
y_slice = adult_data_train_['Target']
X_slice = adult_data_train_.drop(columns=['Target'])

# Garante que as colunas do conjunto de validação estejam alinhadas com o conjunto de treino
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_slice, y_slice)

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print(len(X_slice))
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 542.4702289104462 segundos
28932

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.86 (+/- 0.00)
Recall: 0.63 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.31 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.90
Log-loss: 0.23

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.91      0.96      0.93     22654
           1       0.86      0.71      0.78      7508

    accuracy                           0.90     30162
   macro avg       0.88      0.83      0.86     30162
weighted avg       0.90      0.90      0.89     30162



In [24]:
adult_data_train_, X_train, y_train = import_census()


# Define as métricas de avaliação
scoring = {
    'accuracy': 'accuracy',  
    'recall': 'recall',      
    'f1': 'f1',              
    'log_loss': make_scorer(log_loss, response_method='predict_proba')
}

# Remove aleatoriamente uma quantidade definida de linhas
remove_rows = adult_data_train_.sample(n=len(adult_data_train_.query('Relationship==5 and Race==4').index), random_state=42).index
adult_data_train_.drop(remove_rows, inplace=True)

# Prepara os dados de treino após a remoção
y_slice = adult_data_train_['Target']
X_slice = adult_data_train_.drop(columns=['Target'])

# Garante que as colunas do conjunto de validação estejam alinhadas com o conjunto de treino
X_val = X_val.reindex(columns=X_slice.columns, fill_value=0)

# Treina o modelo no conjunto de treinamento
start_time_model = time.time()
rf_optimized = model_train(X_slice, y_slice)
end_time_model = time.time()
execution_time_model = end_time_model - start_time_model

# Avaliação no conjunto de treinamento usando validação cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results_train = cross_validate(rf_optimized, X_slice, y_slice, cv=skf, scoring=scoring)

# Treina o modelo nos dados de treino completos
rf_optimized.fit(X_slice, y_slice)

# Avaliação no conjunto de validação
y_val_pred = rf_optimized.predict(X_val)
y_val_proba = rf_optimized.predict_proba(X_val)

# Resultados no conjunto de validação
val_accuracy = np.mean(y_val_pred == y_val)
val_log_loss = log_loss(y_val, y_val_proba)

# Exibe os resultados
print("Tempo para treinar o modelo:", execution_time_model, "segundos")
print(len(X_slice))
# Resultados no treinamento (validação cruzada)
print("\nMédia e desvio padrão das métricas no conjunto de treinamento (validação cruzada):")
print(f"Acurácia: {np.mean(results_train['test_accuracy']):.2f} (+/- {np.std(results_train['test_accuracy']):.2f})")
print(f"Recall: {np.mean(results_train['test_recall']):.2f} (+/- {np.std(results_train['test_recall']):.2f})")
print(f"F1 Score: {np.mean(results_train['test_f1']):.2f} (+/- {np.std(results_train['test_f1']):.2f})")
print(f"Log-loss: {np.mean(results_train['test_log_loss']):.2f} (+/- {np.std(results_train['test_log_loss']):.2f})")

# Resultados no conjunto de validação
print("\nResultados no conjunto de validação:")
print(f"Acurácia: {val_accuracy:.2f}")
print(f"Log-loss: {val_log_loss:.2f}")
print("\nRelatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))

Tempo para treinar o modelo: 598.9204413890839 segundos
28973

Média e desvio padrão das métricas no conjunto de treinamento (validação cruzada):
Acurácia: 0.86 (+/- 0.00)
Recall: 0.62 (+/- 0.01)
F1 Score: 0.69 (+/- 0.01)
Log-loss: 0.30 (+/- 0.01)

Resultados no conjunto de validação:
Acurácia: 0.90
Log-loss: 0.23

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.91      0.96      0.93     22654
           1       0.86      0.70      0.77      7508

    accuracy                           0.90     30162
   macro avg       0.88      0.83      0.85     30162
weighted avg       0.89      0.90      0.89     30162

